In [1]:
from tools.utils import reload_module
from datasets.fabric import make_sigmod_ds
import numpy as np
reload_module("datasets")

In [2]:
def test_dataset(name, path, *args, **kwargs):
    ds = make_sigmod_ds(name, *args, **kwargs)
    ds.load(path)
    print(ds.get_data().shape)
    print(ds.get_labels().shape)
    splitted_ds = ds.split_train_val_test(0.8, 0.1)
    print(ds.get_split_indecies()[0].shape)
    print(len(np.unique(ds.get_split_indecies()[0])))
    print(len(splitted_ds.train.labels))
    print(len(splitted_ds.val.labels))
    print(len(splitted_ds.test.labels))

In [3]:
# test_dataset("radioml_2016", "data/radioml_2016/RML2016.10a_dict.pkl")
# test_dataset(
#     "matlab_v2", "data/simc_v2_30k_fpm_0_30_snr/", frames_per_modulation=30_000, snrs=range(0, 30)
# )

In [4]:
radioml_ds = make_sigmod_ds("radioml_2016")
radioml_ds.load("data/radioml_2016/RML2016.10a_dict.pkl")

In [5]:
# from models.fabric import make_sigmod_model, ConvolutionConfiguration

# cnn_v1_configuration = ConvolutionConfiguration(
#     input_shape=(1, 128, 2),
#     n_classes=len(radioml_ds.get_modulations()),
#     output_channels=[32, 48, 64, 96, 128, 192],
#     kernel_sizes=[8, 8, 8, 8, 8, 8],
#     paddings=["same", "same", "same", "same", "same", "same"],
#     max_pool_sizes=[1, 1, 2, 1, 2, 1],
#     max_pool_strides=[1, 1, 2, 1, 2, 1],
#     avg_size=32,
#     dense_sizes=[],
# )

# model = make_sigmod_model("cnn_1d_v1", cnn_v1_configuration)

# model.summary()
# splitted_radioml_ds = radioml_ds.split_train_val_test(0.8, 0.1)
# print(splitted_radioml_ds.train.data[0].shape)

In [6]:
from models.fabric import make_sigmod_model, EncoderTransformerConfiguration, EncoderLayerConfiguration

enc_v1_configuration = EncoderTransformerConfiguration(
    input_shape=(128, 2),
    n_classes=len(radioml_ds.get_modulations()),
    
    cnn_output_channels=[32],
    cnn_kernel_sizes=[8],
    cnn_paddings=["same"],
    
    encoder_layer=EncoderLayerConfiguration(
        h=4,
        d_k=32,
        d_v=32,
        d_model=32,
        d_ff=128,
        n=4,
    ),
    
    avg_size=32,
    dense_sizes=[128],
)

model = make_sigmod_model("encoder_transformer_1d_v1", enc_v1_configuration)
model.summary()
splitted_radioml_ds = radioml_ds.split_train_val_test(0.8, 0.1)
print(splitted_radioml_ds.train.data[0].shape)

2023-06-29 22:55:55.088905: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 22:55:55.113627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 22:55:55.509367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-29 22:55:55.891582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 2)]          0         
                                                                 
 CNN0_ (Conv1D)              (None, 128, 32)           544       
                                                                 
 RELU0_ (ReLU)               (None, 128, 32)           0         
                                                                 
 ENC1_ (SigModEncoder)       (None, 128, 32)           50816     
                                                                 
 AVG1_ (AveragePooling1D)    (None, 4, 32)             0         
                                                                 
 FLT1_ (Flatten)             (None, 128)               0         
                                                                 
 FC0_ (Dense)                (None, 128)               16512 

In [7]:
# TODO: remove this inconsistency
splitted_radioml_ds.train.data = np.squeeze(splitted_radioml_ds.train.data)
splitted_radioml_ds.val.data = np.squeeze(splitted_radioml_ds.val.data)
splitted_radioml_ds.test.data = np.squeeze(splitted_radioml_ds.test.data)

print(splitted_radioml_ds.train.data.shape)

(176000, 128, 2)


In [8]:
import tensorflow as tf

def step_decay(epoch):
    lrate = 0.001
    factor = epoch // 8
    lrate /= 10**factor
    return lrate


lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

model.compile(
    # optimizer=tf.keras.optimizers.Adam(learning_rate=0.0),
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

# N_EPOCHS = 16
# BATCH_SIZE = 256
N_EPOCHS = 40
BATCH_SIZE = 128

h = model.fit(
    splitted_radioml_ds.train.data,
    splitted_radioml_ds.train.labels,
    epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(splitted_radioml_ds.val.data, splitted_radioml_ds.val.labels),
    callbacks=[lrate]
)


Epoch 1/40


2023-06-29 22:56:01.197400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-29 22:56:01.708868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-29 22:56:01.799191: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f99e1902830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-29 22:56:01.799207: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti Laptop GPU, Compute Capability 8.6
2023-06-29 22:56:01.802279: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-29 22:56:01.882747: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lif

1375/1375 [==============================] - 44s 27ms/step - loss: 1.7908 - accuracy: 0.3188 - val_loss: 1.4164 - val_accuracy: 0.4440 - lr: 0.0010
Epoch 2/40
1375/1375 [==============================] - 36s 26ms/step - loss: 1.3632 - accuracy: 0.4705 - val_loss: 1.3747 - val_accuracy: 0.4714 - lr: 0.0010
Epoch 3/40
1375/1375 [==============================] - 37s 27ms/step - loss: 1.2942 - accuracy: 0.5035 - val_loss: 1.2816 - val_accuracy: 0.5157 - lr: 0.0010
Epoch 4/40
1375/1375 [==============================] - 37s 27ms/step - loss: 1.2407 - accuracy: 0.5250 - val_loss: 1.2361 - val_accuracy: 0.5315 - lr: 0.0010
Epoch 5/40
1375/1375 [==============================] - 38s 27ms/step - loss: 1.2144 - accuracy: 0.5364 - val_loss: 1.1896 - val_accuracy: 0.5451 - lr: 0.0010
Epoch 6/40
1375/1375 [==============================] - 38s 27ms/step - loss: 1.1957 - accuracy: 0.5452 - val_loss: 1.1906 - val_accuracy: 0.5436 - lr: 0.0010
Epoch 7/40
1375/1375 [==============================] - 3